# **The Tokenizers**
**Train** a brand **new tokenizer on a corpus of texts**, so it can then be **used to pretrain a language model**. This will all be done with the help of the 🤗 `Tokenizers` library, which provides the **“fast”** tokenizers in the 🤗 Transformers library. We’ll take a close look at the features that this library provides, and explore how the fast tokenizers differ from the **“slow”** versions.

**Topics:**
- How to **train a new tokenizer** similar to the one used by a given checkpoint on a new corpus of texts
- The special features of **fast tokenizers**
- The differences between the **three main subword tokenization algorithms** used in NLP today
- How to **build a tokenizer from scratch** with the 🤗 Tokenizers library and train it on some data.

## **Training a New Tokenizer from an old one**
**If a language model is not available in the language you are interested in**, *or* **if your corpus is very different from the one your language model was trained on**, you will most likely want to **retrain the model from scratch using a tokenizer adapted to your data**. That will require training a new tokenizer on your dataset.


Most Transformers models use a *`subword tokenization algorithm`*. To identify which subwords are of interest and occur most frequently in the corpus at hand, the **`tokenizer` needs to take a hard look at all the texts in the corpus** — a process we call *training*. The exact rules that govern this training depend on **the type of tokenizer used** (*go over 3 main algorithm*).

⚠️ **Training a tokenizer** is not the same as training a model! Model training uses *stochastic gradient descent* to make the loss a little bit smaller for each batch. It’s randomized by nature (meaning you have to set some seeds to get the same results when doing the same training twice).

⚠️ **Training a tokenizer** is a statistical process that tries to **identify which subwords are the best to pick for a given corpus**, *and* **the exact rules used to pick them depend on the tokenization algorithm**. It’s deterministic, meaning you *always get the same results when training with the same algorithm on the same corpus*.

### **Assembling a corpus**
There’s a very simple API in 🤗 Transformers that you can use to **train a new tokenizer with the same characteristics as an existing one**: `AutoTokenizer.train_new_from_iterator()`.

To see this in action, let’s say we want to **train `GPT-2` from scratch, but in a language other than *English*.** Our *first task* will be to **gather lots of data in that language in a training corpus**. To provide examples everyone will be able to understand, we won’t use a language like Russian or Chinese here, but rather a specialized English language: ***Python code.***

The 🤗 Datasets library can help us assemble a corpus of Python source code. We’ll use the usual `load_dataset()` function to download and cache the `CodeSearchNet` dataset. This dataset was created for the CodeSearchNet challenge and **contains millions of functions from open source libraries on GitHub in several programming languages**. Here, we will load the **Python** part of this dataset:

In [ ]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

In [ ]:
raw_datasets["train"]

## Output
# Dataset({
#     features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language',
#       'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name',
#       'func_code_url'
#     ],
#     num_rows: 412178
# })

In [ ]:
raw_datasets["train"][0]

Here. we’ll just use the `whole_func_string` column to train our tokenizer.

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

## Output
# def handle_simple_responses(
#       self, timeout_ms=None, info_cb=DEFAULT_MESSAGE_CALLBACK):
#     """Accepts normal responses from the device.

#     Args:
#       timeout_ms: Timeout in milliseconds to wait for each response.
#       info_cb: Optional callback for text sent from the bootloader.

#     Returns:
#       OKAY packet's message.
#     """
#     return self._accept_responses('OKAY', info_cb, timeout_ms=timeout_ms)

The first thing we need to do is **transform the dataset into an *iterator* of lists of texts** — for instance, a list of list of texts.

Using lists of texts will enable our tokenizer to go faster (training on batches of texts instead of processing individual texts one by one), and it should be an iterator if we want to avoid having everything in memory at once.

If your corpus is huge, you will want to take advantage of the fact that *🤗 `Datasets` does not load everything into RAM but stores the elements of the dataset on disk*.

Create **list of lists of 1000 texts each** by using a *Python generator*, we can avoid Python loading anything into memory until it’s actually necessary.

In [ ]:
# Don't uncomment the following line unless your dataset is small!
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

In [ ]:
## Using Python Generator
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)
# This code doesn’t fetch any elements of the dataset;
# it just creates an object you can use in a Python for loop

The texts will only be loaded when you need them (that is, when you’re at the step of the `for` loop that requires them), and only 1,000 texts at a time will be loaded.

The problem with a *generator object* is that it can only be *used once*. So, instead of this giving us the list of the first 10 digits twice, we get them **once and then an empty list**.

In [ ]:
## Define the function to returns a generator instead
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
# or define a generator inside a for loop by using the yield
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

# which will produce the exact same generator as before,
# but allows you to use more complex logic than you can in a list comprehension.

### **Training a new Tokenizer**
Now that we have our **corpus** in the form of an *iterator of batches of texts*, we are ready to train a new tokenizer.

Load the tokenizer that want to pair with our model (etc., GPT-2)

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch.

This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; **our new tokenizer will be exactly the same as GPT-2**, *and* **the only thing that will change is the vocabulary**, which will be **determined by the training on our corpus**.

In [ ]:
## Example
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

# ['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo',
#  'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']

This tokenizer has a few special symbols, like `Ġ` and `Ċ`, which denote *spaces* and *newlines*, respectively.

As we can see, this is not too efficient: the tokenizer returns individual tokens for each space, when it could group together indentation levels (since having sets of four or eight spaces is going to be very common in code). It also split the function name a bit weirdly, not being used to seeing words with the `_` character.

Let’s train a new tokenizer and see if it solves those issues. For this, we’ll use the method `train_new_from_iterator()`

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

# ['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`',
#  'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']

Here we again see the special symbols `Ġ` and `Ċ` that denote *spaces* and *newlines*, but we can also see that our tokenizer learned some tokens that are highly specific to a corpus of Python functions: for example, there is a `ĊĠĠĠ` token that represents an *indentation*, and a `Ġ"""` token that represents the *three quotes that start a docstring*. The tokenizer also correctly split the function name on `_`.

This is quite a compact representation; comparatively, using the plain English tokenizer on the same example will give us a longer sentence:

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

In [ ]:
# Another example
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

# ['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',',
#  'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_',
#  'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch', '.', 'zeros', '(',
#  'output', '_', 'size', ')', 'ĊĊĠĠĠ', 'Ġdef', 'Ġ__', 'call', '__(', 'self', ',', 'Ġx', '):', 'ĊĠĠĠĠĠĠĠ',
#  'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'bias', 'ĊĠĠĠĠ']

In addition to the token corresponding to an indentation, here we can also see a token for a double indentation: `ĊĠĠĠĠĠĠĠ`. The special Python words like `class`, `init`, `call`, `self`, and `return` are each tokenized as one token, and we can see that as well as splitting on `_` and `.` the tokenizer correctly splits even camel-cased names: LinearLayer is tokenized as `["ĠLinear", "Layer"].`

### **Saving the Tokenizer**
To make sure we can use it later, we need to save our new tokenizer. Like for models, this is done with the `save_pretrained()` method

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

This will create a new folder named `code-search-net-tokenizer`, which will contain all the files the tokenizer needs to be reloaded.

If you want to share this tokenizer with your colleagues and friends, you can upload it to the Hub by logging into your account

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Working with terminal
# huggingface-cli login

**Push tokenizer to the Hub**

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

This will create a new repository in your namespace with the name `code-search-net-tokenizer`, containing the tokenizer file.

In [ ]:
## Load our new Tokenizer
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

## **Fast Tokenizers' special powers**

## **Fast Tokenizers in the `QA Pipeline`**

## **`Normalization` and Pre-Tokenization**

## **`Byte-Pair Encoding` Tokenization**

## **`WordPiece` Tokenization**

## **`Unigram` Tokenization**

## **Building a Tokenizer, block-by-block**